In [ ]:
%pip install --upgrade langchain langchain-community langchain-openai langchain-core langchainhub tavily-python python-dotenv -q


Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import subprocess
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain import hub

# 환경 변수 로드 (.env 파일이 있다면)
load_dotenv()

# Windows에서 setx로 설정한 환경 변수는 새 프로세스에서만 적용됩니다
# Jupyter Notebook이 이미 실행 중이었다면 아래 코드로 다시 로드
try:
    result = subprocess.run(
        ['powershell', '-Command', '[System.Environment]::GetEnvironmentVariable("TAVILY_API_KEY", "User")'],
        capture_output=True,
        text=True
    )
    if result.stdout.strip():
        os.environ["TAVILY_API_KEY"] = result.stdout.strip()
        print("✅ 시스템 환경 변수에서 Tavily API 키를 로드했습니다.")
except:
    pass

# API 키 확인
tavily_api_key = os.getenv("TAVILY_API_KEY", "")
openai_api_key = os.getenv("OPENAI_API_KEY", "")

print("✅ 환경 설정 완료")
print(f"Tavily API Key: {'✅ 설정됨' if tavily_api_key else '❌ 미설정'}")
print(f"OpenAI API Key: {'✅ 설정됨' if openai_api_key else '❌ 미설정'}")



✅ 시스템 환경 변수에서 Tavily API 키를 로드했습니다.
✅ 환경 설정 완료
Tavily API Key: ✅ 설정됨
OpenAI API Key: ✅ 설정됨


In [6]:
# Tavily 검색 Tool 생성
tavily_tool = TavilySearchResults(
    api_key=os.getenv("TAVILY_API_KEY", ""),
    max_results=5
)

print("✅ Tavily Tool 초기화 완료")


✅ Tavily Tool 초기화 완료


In [12]:
# LLM 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=openai_api_key,
    temperature=0
)

# Tools
tools = [tavily_tool]

# Agent 프롬프트 가져오기
prompt = hub.pull("hwchase17/openai-functions-agent")

# Agent 생성
agent = create_openai_tools_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True
)

print("✅ AgentExecutor 생성 완료")



c:\Users\dohom\anaconda3\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.


✅ AgentExecutor 생성 완료


In [15]:
# 질문 1: 날씨 정보
question1 = "내일 서울 비 와?"

print("=" * 60)
print(f"질문: {question1}")
print("=" * 60)
print()

result1 = agent_executor.invoke({"input": question1})

print("\n" + "=" * 60)
print("최종 답변:")
print("=" * 60)
print(result1["output"])


질문: 내일 서울 비 와?



> Entering new AgentExecutor chain...


ModuleNotFoundError: No module named 'langchain_core.beta'

In [ ]:
# 질문 2: 맛집 추천
question2 = "강남역 최신 맛집 3개 추천해줘"

print("=" * 60)
print(f"질문: {question2}")
print("=" * 60)
print()

result2 = agent_executor.invoke({"input": question2})

print("\n" + "=" * 60)
print("최종 답변:")
print("=" * 60)
print(result2["output"])


In [ ]:
# 질문 3: 뉴스 요약
question3 = "NVIDIA 최근 뉴스 요약해줘"

print("=" * 60)
print(f"질문: {question3}")
print("=" * 60)
print()

result3 = agent_executor.invoke({"input": question3})

print("\n" + "=" * 60)
print("최종 답변:")
print("=" * 60)
print(result3["output"])
